# Diffusive toggle switch

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
from pysb.integrate import odesolve
from scripts.models.enzymatic_futile_cycle_pysb import model

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

plt.style.use("./scripts/output/notebooks/custom_style.mplstyle")
%matplotlib inline

In [ ]:
concentrations_ode = odesolve(model, np.arange(1.0))
slice_vec = []
for o in model.observables:
    slice_vec.append(int(np.round(concentrations_ode[o.name][-1])))
slice_vec = np.array(slice_vec)

## Load initial data

Matrix integrator

### TTN integrator

#### Matrix

In [ ]:
tree = readTree("output/efc_r5_e_tau1e-4/output_t100000.nc")
_, ttn_marginal_r_5 = tree.calculateObservables(np.zeros(6, dtype="int"))

tree = readTree("output/efc_r8_e_tau1e-4/output_t100000.nc")
_, ttn_marginal_r_8 = tree.calculateObservables(np.zeros(6, dtype="int"))

tree = readTree("output/efc_r15_e_tau1e-4/output_t100000.nc")
_, ttn_marginal_r_15 = tree.calculateObservables(np.zeros(6, dtype="int"))

tree = readTree("output/efc_r17_e_tau1e-4/output_t100000.nc")
_, ttn_marginal_r_17 = tree.calculateObservables(np.zeros(6, dtype="int"))

### SSA

In [ ]:
idx_2D = np.array([0, 1])

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/enzymatic_futile_cycle_ssa_1e+05.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
ssa_marginal_1e5, _, _, _ = ssa_1e5_sol.calculateObservables(slice_vec, idx_2D)
ssa_wall_time = 136.4

## Convergence with increasing rank

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,8))
# ax1.plot(np.arange(tree.grid.n[0]), ttn_marginal_r_5[0], "v", label="TTN, $r=5$")
ax1.plot(np.arange(tree.grid.n[0]), ttn_marginal_r_8[0], "v", label="TTN, $r=8$")
# ax1.plot(np.arange(tree.grid.n[0]), ttn_marginal_r_15[0], "v", label="TTN, $r=15$")
ax1.plot(np.arange(tree.grid.n[0]), ttn_marginal_r_17[0], "v", label="TTN, $r=17$")
ax1.plot(np.arange(ssa_1e5_sol.n[0])+ssa_1e5_sol.n_min[0], ssa_marginal_1e5[-1][0], "v", label="SSA, $10^5$ runs")
ax1.set_xlabel("$x_0$")

# ax2.plot(np.arange(tree.grid.n[3]), ttn_marginal_r_5[3], "v", label="TTN, $r=5$")
ax2.plot(np.arange(tree.grid.n[3]), ttn_marginal_r_8[3], "v", label="TTN, $r=8$")
ax2.plot(np.arange(tree.grid.n[3]), ttn_marginal_r_15[3], "v", label="TTN, $r=15$")
ax2.plot(np.arange(ssa_1e5_sol.n[1])+ssa_1e5_sol.n_min[1], ssa_marginal_1e5[-1][1], "v", label="SSA, $10^5$ runs")
ax2.set_xlabel("$x_1$")

ax2.legend()
plt.savefig("plots/efc_fig1.pdf")

In [ ]:
time_series = TimeSeries("output/efc_r5_e_tau1e-5")
mass_err_r5 = time_series.getMassErr()

In [ ]:
time = time_series.time

In [ ]:
fig, ax = plt.subplots()
ax.plot(time, np.abs(mass_err_r5), label="TTN, $r=5$")
ax.set_xlabel("$t$")
ax.set_ylabel("$|\Delta m(t)|$")
ax.set_yscale("log")
ax.legend()
plt.savefig("plots/efc_fig2.pdf")